In [60]:
import os
import pandas as pd
import numpy as np
import rasterio
import rasterio.mask
from rasterio.features import rasterize
import geopandas as gpd
from rasterio.windows import Window

In [53]:
directories_to_process = [
    ('/datasets/rpartsey/satellite/planet/SNP_Planet_Scenes_2017_Spring_I', 'spring'),
    ('/datasets/rpartsey/satellite/planet/SNP_Planet_Scenes_2017_Spring_II', 'spring'),
    ('/datasets/rpartsey/satellite/planet/SNP_Planet_Scenes_2017_Summer_Autumn_I', 'summer-autumn')
]

base_dirs_df = pd.DataFrame(directories_to_process, columns=['base_dir', 'tag'])
base_dirs_df.head()

,base_dir,tag
0,/datasets/rpartsey/satellite/planet/SNP_Planet...,spring
1,/datasets/rpartsey/satellite/planet/SNP_Planet...,spring
2,/datasets/rpartsey/satellite/planet/SNP_Planet...,summer-autumn


In [54]:
dfs = []

for i, row in base_dirs_df.iterrows():
    df = pd.read_csv(os.path.join(row.base_dir, 'files.csv'), index_col=0)
    df['base_dir_id'] = i
    dfs.append(df)

files_df = pd.concat(dfs, axis=0, sort=False)
print(files_df.shape)
files_df.head()

(103, 6)


,image,udm_mask,shape_file,xml,json,base_dir_id
20170402_074845_0f17,20170402_074845_0f17_3B_AnalyticMS.tif,20170402_074845_0f17_3B_AnalyticMS_DN_udm.tif,20170402_074845_0f17.shp,20170402_074845_0f17_3B_AnalyticMS_metadata.xml,20170402_074845_0f17_metadata.json,0
20170406_074822_1034,20170406_074822_1034_3B_AnalyticMS.tif,20170406_074822_1034_3B_AnalyticMS_DN_udm.tif,20170406_074822_1034.shp,20170406_074822_1034_3B_AnalyticMS_metadata.xml,20170406_074822_1034_metadata.json,0
20170330_075419_0e2f,20170330_075419_0e2f_3B_AnalyticMS.tif,20170330_075419_0e2f_3B_AnalyticMS_DN_udm.tif,20170330_075419_0e2f.shp,20170330_075419_0e2f_3B_AnalyticMS_metadata.xml,20170330_075419_0e2f_metadata.json,0
20170311_075415_0e0f,20170311_075415_0e0f_3B_AnalyticMS.tif,20170311_075415_0e0f_3B_AnalyticMS_DN_udm.tif,20170311_075415_0e0f.shp,20170311_075415_0e0f_3B_AnalyticMS_metadata.xml,20170311_075415_0e0f_metadata.json,0
20170311_075416_0e0f,20170311_075416_0e0f_3B_AnalyticMS.tif,20170311_075416_0e0f_3B_AnalyticMS_DN_udm.tif,20170311_075416_0e0f.shp,20170311_075416_0e0f_3B_AnalyticMS_metadata.xml,20170311_075416_0e0f_metadata.json,0


In [55]:
merged_df = pd.merge(base_dirs_df, files_df, how='inner', left_index=True, right_on='base_dir_id')
merged_df.head()

,base_dir,tag,image,udm_mask,shape_file,xml,json,base_dir_id
20170402_074845_0f17,/datasets/rpartsey/satellite/planet/SNP_Planet...,spring,20170402_074845_0f17_3B_AnalyticMS.tif,20170402_074845_0f17_3B_AnalyticMS_DN_udm.tif,20170402_074845_0f17.shp,20170402_074845_0f17_3B_AnalyticMS_metadata.xml,20170402_074845_0f17_metadata.json,0
20170406_074822_1034,/datasets/rpartsey/satellite/planet/SNP_Planet...,spring,20170406_074822_1034_3B_AnalyticMS.tif,20170406_074822_1034_3B_AnalyticMS_DN_udm.tif,20170406_074822_1034.shp,20170406_074822_1034_3B_AnalyticMS_metadata.xml,20170406_074822_1034_metadata.json,0
20170330_075419_0e2f,/datasets/rpartsey/satellite/planet/SNP_Planet...,spring,20170330_075419_0e2f_3B_AnalyticMS.tif,20170330_075419_0e2f_3B_AnalyticMS_DN_udm.tif,20170330_075419_0e2f.shp,20170330_075419_0e2f_3B_AnalyticMS_metadata.xml,20170330_075419_0e2f_metadata.json,0
20170311_075415_0e0f,/datasets/rpartsey/satellite/planet/SNP_Planet...,spring,20170311_075415_0e0f_3B_AnalyticMS.tif,20170311_075415_0e0f_3B_AnalyticMS_DN_udm.tif,20170311_075415_0e0f.shp,20170311_075415_0e0f_3B_AnalyticMS_metadata.xml,20170311_075415_0e0f_metadata.json,0
20170311_075416_0e0f,/datasets/rpartsey/satellite/planet/SNP_Planet...,spring,20170311_075416_0e0f_3B_AnalyticMS.tif,20170311_075416_0e0f_3B_AnalyticMS_DN_udm.tif,20170311_075416_0e0f.shp,20170311_075416_0e0f_3B_AnalyticMS_metadata.xml,20170311_075416_0e0f_metadata.json,0


## Generate masks

In [46]:
def create_shapes_bitmask(source_meta, shapes, mask_path):
    """
    Creates bit masks for shapes that intersect with raster image coordinates.

    :param source_meta: opened raster image
    :param shapes: iterable with shapes
    :param mask_file_name: file name to store mask on disk
    :return: None
    """
    im_size = (source_meta['height'], source_meta['width'])

    bitmask = rasterize(
        shapes=shapes,
        out_shape=im_size,
        transform=source_meta['transform']
    )

    bitmask_meta = {
        **source_meta,
        'dtype': rasterio.uint8,
        'count': 1
    }

    with rasterio.open(mask_path, 'w', **bitmask_meta) as dest:
        dest.write(bitmask, indexes=1)

        
def generate_masks(df):
    for image_dir_id, row in df.iterrows():
        image_name = row.image.split('.')[0]
        mask_name = '{}_mask'.format(image_name)
        
        mask_path = os.path.join(row.base_dir, image_dir_id, '{}.tif'.format(mask_name))
        image_path = os.path.join(row.base_dir, image_dir_id, row.image)
        shapes_path = os.path.join(row.base_dir, image_dir_id, row.shape_file)
        matadata_path = os.path.join(row.base_dir, image_dir_id, row.json)
        
        print(mask_path)

        try:
            shapes_df = gpd.read_file(shapes_path)
            ud_region_df = gpd.read_file(matadata_path) # usable data polygon
            ud_shapes_df = gpd.overlay(shapes_df, ud_region_df, how='intersection')
            print(
                '\nOriginal df length: {l1}, intersection df length: {l2}\n'.format(
                l1=shapes_df.shape[0], l2=ud_shapes_df.shape[0])
            )
            shapes_df = ud_shapes_df

        except Exception as e:
            print(e)
            continue

        with rasterio.open(image_path) as src:
            shapes_df = shapes_df.to_crs({'init' : src.meta['crs']['init']})

            create_shapes_bitmask(
                source_meta=src.meta,
                shapes=shapes_df.geometry,
                mask_path=mask_path
            )

In [48]:
# NOTE: run with caution, this will regenerate masks 

# generate_masks(merged_df)

## Crop Images

In [56]:
def generate_window_offsets(image_h, image_w, window_h, window_w):
    """
    Returns iterable with window column and row offsets(top left corner).

    :param image_h: height of raster image
    :param image_w: width of raster image
    :param window_h: window height
    :param window_w: window width
    :return: iterable
    """

    def shift(raster_size, window_size):
        return (raster_size % window_size) // 2

    row_coord = -shift(image_h, window_h)
    col_coord = -shift(image_w, window_w)

    rows = np.arange(row_coord, image_h, window_h)
    cols = np.arange(col_coord, image_w, window_w)

    rows, cols = np.meshgrid(rows, cols, indexing='ij')

    return zip(rows.ravel(), cols.ravel())

In [61]:
# NOTE: run with caution, this will recrop images and masks 

# df = merged_df

# window_w = 256
# window_h = 256

# base_dest_dir = '/datasets/rpartsey/satellite/planet/2017-su-au-1_2017-sp-1_2017-sp-2_256x256'
# dest_image_dir = os.path.join(base_dest_dir, 'images')
# dest_mask_dir = os.path.join(base_dest_dir, 'masks')

# #  	base_dir 	tag 	image 	udm_mask 	shape_file 	xml 	json

# for image_dir_id, row in df.iterrows():
    
#     file_name = row.image.split('.')[0]
#     tif_path = os.path.join(row.base_dir, image_dir_id, row.image)
#     mask_path = os.path.join(row.base_dir, image_dir_id, '{}_mask.tif'.format(file_name))
    
#     with rasterio.open(tif_path) as tif_src:        
#         with rasterio.open(mask_path) as mask_src:
            
#             assert tif_src.meta['height'] == mask_src.meta['height'] and tif_src.meta['width'] == mask_src.meta['width'], 'Shapes don\'t match!'
            
#             window_offsets = generate_window_offsets(
#                 image_h=tif_src.meta['height'],
#                 image_w=tif_src.meta['width'],
#                 window_h=window_h,
#                 window_w=window_w
#             )

#             for row_off, col_off in window_offsets:
#                 window = Window(
#                     col_off=col_off,
#                     row_off=row_off,
#                     width=window_w,
#                     height=window_h
#                 )
                
#                 dest_meta = {
#                     **tif_src.meta,
#                     'height': window.height,
#                     'width': window.width,
#                     'transform': rasterio.windows.transform(window, tif_src.transform)
#                 }
                
                
#                 tif_data = tif_src.read(window=window, boundless=True, fill_value=0)
#                 if not tif_data.any():
#                     continue
                
#                 dest_file_name = '{}_{}_{}_{}.tif'.format(file_name, (row_off // window_h) + 1, (col_off // window_w) +1, row.tag)
#                 dest_path = os.path.join(dest_image_dir, dest_file_name)
                
#                 with rasterio.open(dest_path, 'w', **dest_meta) as tif_dest:
#                     tif_dest.write(tif_data)
                    
#                 dest_meta = {
#                     **mask_src.meta,
#                     'height': window.height,
#                     'width': window.width,
#                     'transform': rasterio.windows.transform(window, mask_src.transform)
#                 }

#                 dest_path = os.path.join(dest_mask_dir, dest_file_name)
                
#                 with rasterio.open(dest_path, 'w', **dest_meta) as mask_dest:
#                     mask_dest.write(mask_src.read(window=window, boundless=True, fill_value=0))
                    
#     print(image_dir_id)

20170402_074845_0f17
20170406_074822_1034
20170330_075419_0e2f
20170311_075415_0e0f
20170311_075416_0e0f
20170406_074823_1034
20170329_074622_1024
20170406_074819_1034
20170403_075403_0e16
20170329_074626_1024
20170406_074835_1038
20170406_074833_1038
20170403_075404_0e16
20170329_074627_1024
20170403_075402_0e16
20170330_075418_0e2f
20170406_074824_1034
20170330_075416_0e2f
20170329_074623_1024
20170402_074847_0f17
20170406_074837_1038
20170328_074657_0f1d
20170329_074624_1024
20170330_075417_0e2f
20170328_074654_0f1d
20170402_074846_0f17
20170406_074821_1034
20170329_074625_1024
20170411_074638_1017
20170328_074655_0f1d
20170406_074832_1038
20170328_074656_0f1d
20170406_074836_1038
20170406_074834_1038
20170411_075455_0e30
20170506_074919_1002
20170430_074903_0f52
20170426_074829_0f12
20170427_085049_0c81
20170416_074655_0f52
20170411_075453_0e30
20170507_075529_0e26
20170426_074935_1009
20170505_075404_0e30
20170507_075527_0e26
20170430_074902_0f52
20170426_074828_0f12
20170429_0857